In [7]:
import requests
from bs4 import BeautifulSoup 
import numpy as np
import pandas as pd
from urllib.request import urlopen
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm

In [8]:
DRIVER_PATH = '/Users/ilyasumernikov/Atlas/chromedriver'

In [19]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

<ipython-input-19-4a6b6a84448f>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


In [22]:
shanghai_result = pd.DataFrame(columns=['year', 'world_rank', 'institution', 'country_name', 'total_score', 'award'])

for j in tqdm(range(2003, 2023)):
    universities_final = []
    countries_final = []
    national_number_final = []
    total_score_final = []
    award_final = []
    world_number_final = []
    url = 'https://www.shanghairanking.com/rankings/arwu/' + str(j)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    page_number = int(soup.find_all('li', class_ = 'ant-pagination-item')[-1].get('class')[-1].split('-')[-1])


    for i in range(2, page_number + 2):
        universities_list = soup.find('table', class_ = 'rk-table').find_all('span', class_ = 'tooltiptext shadow-xs')
        universities_results = [universities_list[x].text for x in range(len(universities_list))]
        universities_final += universities_results

        countries_list = soup.find('table', class_ = 'rk-table').find_all('div', class_ = 'region-img')
        countries_results = [str(countries_list[x]).split('/')[-2].split('.')[0] for x in range(len(countries_list))]
        countries_final += countries_results

        number_list = soup.find('table', class_ = 'rk-table').find_all('td', class_ = '')
        national_number_list = soup.find('table', class_ = 'rk-table').find_all('td', class_ = '')
        national_number_results = [' '.join(number_list[x].text.split()) for x in range(2, len(number_list), 5)]
        national_number_final += national_number_results

        total_score_results = [' '.join(number_list[x].text.split()) for x in range(3, len(number_list), 5)]
        total_score_final += total_score_results

        award_results = [' '.join(number_list[x].text.split()) for x in range(4, len(number_list), 5)]
        award_final += award_results

        world_number_list = [' '.join(number_list[x].text.split()) for x in range(0, len(number_list), 5)]
        world_number_final += world_number_list
        try:
            button_class = "ant-pagination-item-" + str(i)
            button = driver.find_element(By.CLASS_NAME, button_class)
            # Click the button
            button.click()
            soup = BeautifulSoup(driver.page_source, 'lxml')
        except:
            pass
    
    for k in range(len(universities_final)):
        shanghai_result = shanghai_result.append({'year': j,
                                          'world_rank': world_number_final[k],
                                          'institution': universities_final[k],
                                          'country_name': countries_final[k],
                                          'total_score': total_score_final[k],
                                          'award': award_final[k]}, 
                                         ignore_index=True)
    

    
    # creating a dataset for each year
    shanghai_result_year = pd.DataFrame(columns=['year', 'world_rank', 'institution', 'country_name', 'total_score', 'award'])
    for m in range(len(universities_final)):
        shanghai_result_year = shanghai_result_year.append({'year': j,
                                          'world_rank': world_number_final[m],
                                          'institution': universities_final[m],
                                          'country_name': countries_final[m],
                                          'total_score': total_score_final[m],
                                          'award': award_final[m]}, 
                                         ignore_index=True)
    
    year_results = 'shanghai_results_' + str(j) + '.csv'
    shanghai_result_year.to_csv(year_results)
    
shanghai_result.to_csv('shanghai_results.csv')

100%|██████████| 20/20 [02:29<00:00,  7.46s/it]
